In [1]:
import pandas as pd

### Задание 1

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
(df[['userId', 'timestamp']]
    .groupby('userId')
    .filter(lambda x: len(x) > 100)
         .groupby('userId')
         .agg(lambda x: max(x) - min(x) + 1)
         .mean() / 60 / 60 / 24 # in days
)

timestamp    463.894774
dtype: float64

### Задание 2

    Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

К домашнему заданию, задача 2

Дана статистика услуг перевозок клиентов компании по типам:

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов



In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
# rzd

In [5]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
# auto

In [6]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
# air

In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
# client_base

In [8]:
(client_base[['client_id']]
     .merge(rzd, how='left', on='client_id')
     .merge(auto, how='left', on='client_id')
     .merge(air, how='left', on='client_id')
     .fillna(0)
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [9]:
(client_base
     .merge(rzd, how='left', on='client_id')
     .merge(auto, how='left', on='client_id')
     .merge(air, how='left', on='client_id')
     .fillna(0)
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


### Задание 3

 В задаче сквозной аналитики вам предоставили данные по местоположению пользователей.
 Т. е. для каждого user_id известна последовательность координат (широта/долгота),
 когда они требовались приложению для полноценной работы.
    
    Как бы вы добавили эти сведения в таблицу визитов и покупок?
    
 Для составления ответа можно использовать вопросы:

 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
 
 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
   
 3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


1. сопоставить место с визитом или фактом покупки через время совершения действия.
2. при значительном разбросе коордианат можно вместо исходных координат использовать средние или медианные значения. также при наличиислучаев очень большого разброса (например, определение положения по IP при использовании proxy) добавлять флаг ненадёжности определения координат.

3. По коорднатам с помощью карты можно примерно проклассифицировать каждое место использования приложения.
Сопоставив со временем использования можно оценить место работы, жилья, оценить график работы, методы передвижения (общцественный трансопрт, такси/личное авто, и т.д.), оценить не находится ли человек на обеденном перерыве в ресторанах, где и проводит совобдное место. Из всех этих данных можно попробовать вытащить информацию о поле/возрасте/достатке/предпочтениях/семейном статусе.